<a href="https://colab.research.google.com/github/Pratishrutiksah/DAS/blob/main/DAS_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Import the data set
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/Weather_Data_Hourly.csv')

In [9]:
df.describe()

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirdegree,weatherCode,precipMM,precipInches,humidity,...,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF
count,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,...,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000,48168.000000
mean,1150.000000,27.916334,82.242505,6.806843,10.953808,207.448057,132.596226,0.062099,0.001758,58.378073,...,32.736983,90.917103,19.138806,66.441538,29.232748,84.578309,9.395034,15.118128,32.584641,90.637664
std,692.225841,5.314407,9.556917,3.534463,5.671304,89.287049,55.623484,0.361414,0.015044,19.520963,...,6.545317,11.773058,6.668760,11.998918,4.690782,8.436255,4.956839,7.966098,6.751236,12.151213
min,0.000000,11.000000,52.000000,0.000000,0.000000,0.000000,113.000000,0.000000,0.000000,7.000000,...,14.000000,58.000000,-3.000000,27.000000,13.000000,56.000000,0.000000,0.000000,13.000000,56.000000
25%,575.000000,25.000000,76.000000,4.000000,7.000000,161.000000,113.000000,0.000000,0.000000,43.000000,...,27.000000,81.000000,14.000000,57.000000,26.000000,80.000000,6.000000,10.000000,27.000000,81.000000
50%,1150.000000,28.000000,83.000000,6.000000,10.000000,198.000000,113.000000,0.000000,0.000000,59.000000,...,33.000000,92.000000,22.000000,71.000000,29.000000,85.000000,8.000000,14.000000,33.000000,92.000000
75%,1725.000000,31.000000,88.000000,9.000000,14.000000,278.000000,116.000000,0.000000,0.000000,75.000000,...,38.000000,100.000000,25.000000,77.000000,32.000000,90.000000,12.000000,19.000000,38.000000,100.000000
max,2300.000000,45.000000,114.000000,35.000000,56.000000,360.000000,389.000000,10.200000,0.400000,98.000000,...,55.000000,131.000000,29.000000,85.000000,47.000000,116.000000,51.000000,82.000000,55.000000,131.000000


In [10]:
df.shape

(48168, 29)

In [3]:
# Install PySpark
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=14394e313b4bccf775561ab9492704f25955422d83262adc323a77e3821044e9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# Create Spark session
spark = SparkSession.builder.appName("WeatherDataML").getOrCreate()

# Load the dataset
data_path = "/content/Weather_Data_Hourly.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Show the dataset
df.show(5)

# Preprocess the data (example: select relevant features and target variable)
# Assuming we are predicting 'Temperature' based on other weather features
features = df.columns
#features.remove('Temperature')

# Define the VectorAssembler
assembler = VectorAssembler(inputCols=features, outputCol="features")


+----------+----+-----+-----+--------------+-------------+-------------+--------------+-----------+--------------------+-----------+--------+------------+--------+------------+---------------+----------+--------------+----------+----------+----------+---------+---------+----------+----------+-------------+------------+----------+----------+
|      date|time|tempC|tempF|windspeedMiles|windspeedKmph|winddirdegree|winddir16point|weatherCode|      weatherIconUrl|weatherDesc|precipMM|precipInches|humidity|visibilityKm|visibilityMiles|pressureMB|pressureInches|cloudcover|HeatIndexC|HeatIndexF|DewPointC|DewPointF|WindChillC|WindChillF|WindGustMiles|WindGustKmph|FeelsLikeC|FeelsLikeF|
+----------+----+-----+-----+--------------+-------------+-------------+--------------+-----------+--------------------+-----------+--------+------------+--------+------------+---------------+----------+--------------+----------+----------+----------+---------+---------+----------+----------+-------------+-------

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline



# Preprocess the data (example: select relevant features and target variable)
# Assuming we are predicting 'Temperature' based on other weather features
# Get the column names and convert them to lowercase for consistency
features = [col.lower() for col in df.columns]
# Check if 'temperature' exists in lowercase and remove it
if 'temperature' in features:
    features.remove('temperature')
    # Identify string and date columns
    string_columns = [f for f in features if df.schema[f].dataType == 'string']
    date_columns = [f for f in features if df.schema[f].dataType == 'date']

    # Handle categorical features (StringIndexer and OneHotEncoder)
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in string_columns]
    encoders = [OneHotEncoder(inputCol="{0}_indexed".format(c), outputCol="{0}_encoded".format(c)) for c in string_columns]

    # Handle date features (you might need to extract features like day, month, year)
    # For simplicity, let's drop the date column for now
    features = [f for f in features if f not in date_columns]

    # Update features to include encoded categorical features
    features += [encoder.getOutputCol() for encoder in encoders]

    # Define the VectorAssembler after preprocessing
    assembler = VectorAssembler(inputCols=features, outputCol="features")

    # Create the pipeline with preprocessing steps
    pipeline_stages = indexers + encoders + [assembler]

    # Create the pipeline
    pipeline = Pipeline(stages=pipeline_stages)

    # Fit the pipeline to the dataset
    pipeline_model = pipeline.fit(df)

    # Transform the dataset
    df_transformed = pipeline_model.transform(df)

    # Select the relevant columns
    df_final = df_transformed.select("features", col("temperature").alias("label"))

    # Split the data into training and test sets
    train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)
else:
    print("Column 'temperature' not found in the dataset. Please check the column names.")

Column 'temperature' not found in the dataset. Please check the column names.


In [ ]:
# Create and fit the Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol='label')

# Fit the assembler to the training data to create the 'features' column
train_data_with_features = assembler.transform(train_data)

# Now fit the Linear Regression model on the transformed training data
lr_model = lr.fit(train_data_with_features)

# Make predictions on the test data (also transformed)
test_data_with_features = assembler.transform(test_data)
lr_predictions = lr_model.transform(test_data_with_features)

# Evaluate the model
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")
lr_rmse = lr_evaluator.evaluate(lr_predictions)
print(f"Linear Regression RMSE: {lr_rmse}")

In [ ]:
# Create and fit the Decision Tree Regressor model
dt = DecisionTreeRegressor(featuresCol='features', labelCol='label')
dt_model = dt.fit(train_data)

# Make predictions
dt_predictions = dt_model.transform(test_data)

# Evaluate the model
dt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")
dt_rmse = dt_evaluator.evaluate(dt_predictions)
print(f"Decision Tree Regression RMSE: {dt_rmse}")


In [ ]:
# Create and fit the Random Forest Regressor model
rf = RandomForestRegressor(featuresCol='features', labelCol='label')
rf_model = rf.fit(train_data)

# Make predictions
rf_predictions = rf_model.transform(test_data)

# Evaluate the model
rf_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")
rf_rmse = rf_evaluator.evaluate(rf_predictions)
print(f"Random Forest Regression RMSE: {rf_rmse}")
